In [ ]:
# Importing python packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os 

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

# controls default text sizes
plt.rc('lines',markersize=8)
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
# Importing files
REPO = 'viscosity_liquid_transfer_Pablo'
folder = os.getcwd().split(REPO)[0]+REPO+r'\Opentrons_experiments\\BOTorch_optimization\\VS_code_csv\\'
std = '204'


In [ ]:
#Load data frame
folder1 = os.getcwd().split(REPO)[0]+REPO+r'\Opentrons_experiments\\Std_calibrations\\'
df_man = pd.read_csv(folder1+r'\Viscosity_std_'+std+'.csv')

df_man=df_man.drop_duplicates(subset=["volume","aspiration_rate","dispense_rate","blow_out_rate","delay_aspirate","delay_dispense","delay_blow_out","touch_tip_aspirate","touch_tip_dispense"], keep='first')
df_man['time_for_1000']=0
for i in range(len(df_man)):
    if (df_man['blow_out_rate'].iloc[i])<=0:
        df_man['time_for_1000'].iloc[i] = 1000/df_man["aspiration_rate"].iloc[i] + 1000/df_man["dispense_rate"].iloc[i] + df_man["delay_aspirate"].iloc[i] + df_man["delay_dispense"].iloc[i] + df_man["delay_blow_out"].iloc[i]
    else:
        df_man['time_for_1000'].iloc[i] = 1000/df_man["aspiration_rate"].iloc[i] + 1000/df_man["dispense_rate"].iloc[i] + 150/df_man["blow_out_rate"].iloc[i] + df_man["delay_aspirate"].iloc[i] + df_man["delay_dispense"].iloc[i] + df_man["delay_blow_out"].iloc[i]
df_man.reset_index(drop=True)

df_man['iteration']= None
counter= 1
for row in df_man.drop_duplicates(["aspiration_rate","dispense_rate","blow_out_rate","delay_aspirate","delay_dispense","delay_blow_out","touch_tip_aspirate","touch_tip_dispense"]).iterrows():
    df_repeat = df_man.loc[:,["aspiration_rate","dispense_rate","blow_out_rate","delay_aspirate","delay_dispense","delay_blow_out","touch_tip_aspirate","touch_tip_dispense"]]== row[1].loc[["aspiration_rate","dispense_rate","blow_out_rate","delay_aspirate","delay_dispense","delay_blow_out","touch_tip_aspirate","touch_tip_dispense"]]
    index = df_repeat.where(df_repeat.sum(axis=1)==8).dropna().index
    df_man.loc[index,'iteration']=counter
    counter+=1
df_man = df_man.sort_values('iteration')
df_man

#Calcualte means
df_duplicates = df_man.where(df_man.duplicated('iteration',keep=False)==True).dropna(how='all')
df_incomplete = df_man.where(df_man.duplicated('iteration',keep=False)==False).dropna(how='all')
df_mean = pd.DataFrame(columns= df_man.columns)

for index,values in df_duplicates.drop_duplicates(['aspiration_rate','dispense_rate']).iterrows():
    if len(df_duplicates.loc[index:index+2]) == 3:
        mean_error =df_duplicates.loc[index:index+2,'%error'].mean()
        df_duplicates.loc[index,'%error'] = mean_error
        df_duplicates.loc[index, 'volume'] ='mean'+str([300, 500, 1000])
        df_mean = pd.concat([df_mean,df_man.loc[index:index+2],df_duplicates.loc[[index]]])
    else:
        df_incomplete = pd.concat([df_incomplete,df_duplicates.loc[index:index+2]]).drop_duplicates()
df_mean = pd.concat([df_mean,df_incomplete])
df_mean = df_mean.sort_values('iteration')
#df_man = df_mean.reset_index(drop=True)   
"""
#Calculate iteration 
counter =1 
for row in df_man.drop_duplicates(['aspiration_rate','dispense_rate']).iterrows():
    df_repeat = df_man.loc[:,['aspiration_rate','dispense_rate']]== row[1].loc[['aspiration_rate','dispense_rate']]
    index = df_repeat.where(df_repeat.sum(axis=1)==2).dropna().index
    df_man.loc[index,'iteration']=counter
    counter+=1
"""

In [ ]:
df_man

In [ ]:
df_mean

In [ ]:
#Load dataframe
df_exp2 = pd.read_csv(folder+r'\Viscosity_std_'+std+'_3_vol_opt.csv')

#Calculate iterations
counter =int(1) 
for i in range(0,len(df_exp2)+1,3):
    if len(df_exp2.loc[i:i+2,'volume'].drop_duplicates())==1:
        df_exp2.loc[i,'iteration']=int(counter)
        counter+=1    
        df_exp2.loc[i+1,'iteration']=int(counter)
        counter+=1  
        df_exp2.loc[i+2,'iteration']=int(counter)
        counter+=1          
    else:
        df_exp2.loc[i:i+2,'iteration']=int(counter)
        counter+=1



In [ ]:
df_exp2

In [ ]:
#Load data frame
df_exp3 = pd.read_csv(folder+r'\Viscosity_std_'+std+'_exp3_withacq.csv')

#Calculate iteration number
counter =1 
for row in df_exp3.drop_duplicates(['aspiration_rate','dispense_rate']).iterrows():
    df_repeat = df_exp3.loc[:,['aspiration_rate','dispense_rate']]== row[1].loc[['aspiration_rate','dispense_rate']]
    index = df_repeat.where(df_repeat.sum(axis=1)==2).dropna().index
    df_exp3.loc[index,'iteration']=int(counter)
    counter+=1

#Edit volumes to be all the same 
volumes = ['1000.0','500.0','300.0']
for volume in volumes:
    index = df_exp3.where(df_exp3.volume==volume).dropna(how='all').index
    df_exp3.loc[index,'volume'] = volume[:-2]

#Recalculate mean
for i in range(3,len(df_exp3),4):
    df_exp3.loc[i,'%error'] = df_exp3.loc[i-3:i,'%error'].mean()




In [ ]:
df_exp3

#### Plot for exp2 and manual calibration

In [ ]:
#Makes a figure with manual and ML calibrations, one per volume

for volume in [1000,500,300]:
    df_exp2_plot = df_exp2.where(df_exp2.volume==volume).dropna(how='all').copy()
    df_man_plot = df_man.where(df_man.volume==volume).dropna(how='all').copy()
    #Making subplots (2 rows, 1 column)
    fig,axs = plt.subplots(1,2)

    # Plotting graph 1: Iteration against error
    if volume == 1000:
        color = 'red'
    elif volume== 500:
        color = 'green'
    elif volume ==300:
        color = 'black'    
    
    if len(df_man) != 0:
        axs.flatten()[0].scatter(df_man_plot['iteration'],df_man_plot['%error'], marker= 'o', label = str(volume)+u' \u03bcL', c = color)
        axs.flatten()[0].plot(df_man_plot['iteration'],df_man_plot['%error'], label = 'Human Driven', c = 'cornflowerblue')
   
    if len(df_exp2) != 0:
        axs.flatten()[0].scatter(df_exp2_plot['iteration'],df_exp2_plot['%error'], marker= 'o', c = color)
        axs.flatten()[0].plot(df_exp2_plot['iteration'],df_exp2_plot['%error'], label = 'ML Driven', c = 'orangered')
    
    
    axs.flatten()[0].axhline(y=0, ls = '--', color = 'black', linewidth=0.5)
    axs.flatten()[0].axhline(y=2.5, ls = '--', color = 'black', linewidth=0.5)
    axs.flatten()[0].axhline(y=-2.5, ls = '--', color = 'black', linewidth=0.5)
    axs.flatten()[0].set_xlabel('Iteration')
    axs.flatten()[0].set_ylabel('Error [%]')



    axs.flatten()[1].scatter(df_exp2_plot['iteration'],df_exp2_plot['time_asp_1000'], marker= 'o', c = color)
    axs.flatten()[1].plot(df_exp2_plot['iteration'],df_exp2_plot['time_asp_1000'],  c = 'orangered')

    axs.flatten()[1].scatter(df_man_plot['iteration'],df_man_plot['time_for_1000'], marker= 'o', c = color)
    axs.flatten()[1].plot(df_man_plot['iteration'],df_man_plot['time_for_1000'], c = 'cornflowerblue')


    axs.flatten()[1].set_xlabel('Iteration')
    axs.flatten()[1].set_ylabel('Time for 1000ul (s)')

    fig.set_size_inches(15, 4)
    fig.suptitle('Viscosity_std_'+std+'_'+'_exp2_CCF_initialisation')
    fig.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',ncol=5, mode="expand", borderaxespad=0.)
    fig.tight_layout()

    REPO = 'viscosity_liquid_transfer_Pablo'
    #folder = os.getcwd().split(REPO)[0]+REPO+r'\Sartorious_experiments\Finalised_Data'
    # fig.savefig(folder+r'/'+'Viscosity_std_'+std+'_exp2_line.png')

In [ ]:
#Making subplots (2 rows, 2 column)
fig,axs = plt.subplots(1,2)

# Dropping NAN rows
df_man_1000 = df_man.where(df_man.volume==1000).dropna(how='all')
df_man_500 = df_man.where(df_man.volume==500).dropna(how='all')
df_man_300 = df_man.where(df_man.volume==300).dropna(how='all')
df_man_100 = df_man.where(df_man.volume==100).dropna(how='all')


df_exp3_1000 = df_exp3.where(df_exp3.volume=='1000').dropna(how='all')
df_exp3_500 = df_exp3.where(df_exp3.volume=='500').dropna(how='all')
df_exp3_300 = df_exp3.where(df_exp3.volume=='300').dropna(how='all')
df_exp3_100 = df_exp3.where(df_exp3.volume=='100').dropna(how='all')
df_exp3_mean = df_exp3.where(df_exp3.volume=='mean[300, 500, 1000]').dropna(how='all')

#find min error

df_exp3_mean_min_iteration = df_exp3_mean[['iteration','%error']].abs().sort_values('%error').iloc[0:3,0].copy()
df_exp3_mean_min_iteration_df = pd.DataFrame(data=df_exp3_mean_min_iteration)

test_mean_df = pd.DataFrame()
test_300_df = pd.DataFrame()
test_500_df = pd.DataFrame()
test_1000_df = pd.DataFrame()
for i in df_exp3_mean_min_iteration_df['iteration']:
    test_mean_df= pd.concat([test_mean_df, df_exp3_mean[df_exp3_mean['iteration']==int(i)]])
    test_300_df= pd.concat([test_300_df, df_exp3_300[df_exp3_300['iteration']==int(i)]])
    test_500_df= pd.concat([test_500_df, df_exp3_500[df_exp3_500['iteration']==int(i)]])
    test_1000_df= pd.concat([test_1000_df, df_exp3_1000[df_exp3_1000['iteration']==int(i)]])

# Plotting graph 1: Iteration against error
if len(df_man) != 0:
    axs.flatten()[0].plot(df_man['iteration'],df_man['%error'],marker= 'o', label = 'Human Driven mean', c = 'cornflowerblue')


if len(df_exp3_mean) != 0:
    axs.flatten()[0].plot(df_exp3_mean['iteration'],df_exp3_mean['%error'],marker= 'o', label='ML Driven mean',c = 'orangered')


for i in range(len(axs.flatten())-1):
    axs.flatten()[i].axhline(y=0, ls = '--', color = 'black', linewidth=0.5)
    axs.flatten()[i].axhline(y=2.5, ls = '--', color = 'black', linewidth=0.5)
    axs.flatten()[i].axhline(y=-2.5, ls = '--', color = 'black', linewidth=0.5)
    axs.flatten()[i].set_xlabel('Iteration')
    axs.flatten()[i].set_ylabel('Error [%]')



if len(df_exp3_mean) != 0:
    axs.flatten()[1].plot(df_exp3_mean['iteration'],df_exp3_mean['time_asp_1000'], marker ='o', c = 'orangered')

axs.flatten()[0].plot(test_300_df['iteration'],test_300_df['%error'], marker= 'x', c = 'black', linestyle='None')
axs.flatten()[0].plot(test_500_df['iteration'],test_500_df['%error'], marker= 'x', c = 'green', linestyle='None')
axs.flatten()[0].plot(test_1000_df['iteration'],test_1000_df['%error'], marker= 'x', c = 'red', linestyle='None')
axs.flatten()[1].plot(df_man['iteration'],df_man['time_for_1000'], marker= 'o', c = 'cornflowerblue')
axs.flatten()[1].set_xlabel('Iteration')
axs.flatten()[1].set_ylabel('Time for 1000ul (s)')


fig.set_size_inches(15, 4)
fig.suptitle('Viscosity standard '+str(std)+ 'cp batch volume optimization')
fig.legend( loc='upper right',ncol=1, borderaxespad=0.)
fig.tight_layout()

REPO = 'viscosity_liquid_transfer_Pablo'
#folder = os.getcwd().split(REPO)[0]+REPO+r'\Sartorious_experiments\Finalised_Data'
# fig.savefig(folder+r'/'+'Viscosity_std_'+std+'_exp2_line.png')





In [ ]:
df_exp3_500.where(df_exp3_500['iteration']==df_exp3_mean_min_iteration)

In [ ]:
df_exp3_mean

In [ ]:
df_exp3_mean_min_iteration = df_exp3_mean[['iteration','%error']].abs().sort_values('%error').iloc[0:3,0].copy()
df_exp3_mean_min_iteration_df = pd.DataFrame(data=df_exp3_mean_min_iteration)
df_exp3_mean_min_iteration_df

In [ ]:
df_man_mean_min_iteration = df_man[['iteration','%error']].abs().sort_values('%error').iloc[0:3,0].copy()
df_man_mean_min_iteration_df = pd.DataFrame(data=df_man_mean_min_iteration)
df_man_mean_min_iteration_df

In [ ]:
df_exp3_mean

In [ ]:
df_man

In [ ]:
test_mean_df = pd.DataFrame()
test_300_df = pd.DataFrame()
test_500_df = pd.DataFrame()
test_1000_df = pd.DataFrame()
for i in df_exp3_mean_min_iteration_df['iteration']:
    test_mean_df= pd.concat([test_mean_df, df_exp3_mean[df_exp3_mean['iteration']==int(i)]])
    test_300_df= pd.concat([test_300_df, df_exp3_300[df_exp3_300['iteration']==int(i)]])
    test_500_df= pd.concat([test_500_df, df_exp3_500[df_exp3_500['iteration']==int(i)]])
    test_1000_df= pd.concat([test_1000_df, df_exp3_1000[df_exp3_1000['iteration']==int(i)]])
test_1000_df

In [ ]:
t=np.arange(0, 5, 0.2)

plt.plot(t**2, "g*")
plt.plot(t, "r--")


plt.xlabel("Time")
plt.suptitle("Superimposing both graphs in one")
plt.show()